In [2]:
# import libraries
import sqlalchemy
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy import inspect

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

In [3]:
# connecting to the Database using SQLAlchemy
engine = create_engine("sqlite:///Northwind_small.sqlite")

# create an inspector object
inspector = inspect(engine)

# get names of tables in database
print(inspector.get_table_names())

['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


In [4]:
# write a quick function that prints the columns of a table and each column's type

def get_columns_info(table_name):    
    """ Function takes in table name and prints columns' names
    and datatypes """
    
    cols_list = inspector.get_columns(table_name)    
    print(f"Table Name: {table_name}\n")
    for col in cols_list:
        print(f"Name: {col['name']} \t Type: {col['type']}")

In [5]:

# calling the function to test it
get_columns_info('OrderDetail')

Table Name: OrderDetail

Name: Id 	 Type: VARCHAR(8000)
Name: OrderId 	 Type: INTEGER
Name: ProductId 	 Type: INTEGER
Name: UnitPrice 	 Type: DECIMAL
Name: Quantity 	 Type: INTEGER
Name: Discount 	 Type: FLOAT


In [6]:

# joining "Supplier", "Product", "OrderDetail", "Order" and "Customer" tables 
# using the pd.read_sql_query() method and giving them meaningful column names in that step

df1 = pd.read_sql_query('''SELECT p.Id product_id, p.ProductName product_name, s.Id supplier_id, 
                            s.Country supplier_country, s.Region supplier_region, od.Quantity order_quantity,
                            od.UnitPrice unit_price, c.Id customer_id, c.Country customer_country, 
                            c.Region customer_region
                            FROM Supplier s
                            JOIN Product p ON s.Id = p.SupplierId
                            JOIN OrderDetail od ON p.Id = od.ProductId
                            JOIN [Order] o ON od.OrderId = o.Id
                            JOIN Customer c ON o.CustomerId = c.Id
                            ;''', engine)

In [7]:
df1.head()

,product_id,product_name,supplier_id,supplier_country,supplier_region,order_quantity,unit_price,customer_id,customer_country,customer_region
0,11,Queso Cabrales,5,Spain,Southern Europe,12,14.0,VINET,France,Western Europe
1,42,Singaporean Hokkien Fried Mee,20,Singapore,South-East Asia,10,9.8,VINET,France,Western Europe
2,72,Mozzarella di Giovanni,14,Italy,Southern Europe,5,34.8,VINET,France,Western Europe
3,14,Tofu,6,Japan,Eastern Asia,9,18.6,TOMSP,Germany,Western Europe
4,51,Manjimup Dried Apples,24,Australia,NSW,40,42.4,TOMSP,Germany,Western Europe
